In [1]:
from pathlib import Path
import frads as fr
from frads import methods, parsers
import matplotlib.pyplot as plt
import numpy as np
import rbfopt

## Read the configuration file

In [2]:
config = parsers.parse_mrad_config(Path("three_phase.cfg"))

## Assemble the model and generate the matrices
Note: the "no_multiply" option is set to True in the configuration file

In [3]:
with methods.assemble_model(config) as model:
    mtx_paths = methods.three_phase(model, config)

## Load view and daylight matrices into memory

In [5]:
matrices = {'vmx': {}, 'dmx': {}}
for i in range(1, 49):
    matrices['vmx'][i] = fr.load_matrix(mtx_paths.pvmx[f'grids_glazing.{i}'])
    matrices['dmx'][i] = fr.load_matrix(mtx_paths.dmx[f's_glazing.{i}'])

## Load blinds BSDF into memory

In [6]:
bsdf = {}
for i in range(0, 90, 10):
    bsdf[i] = fr.load_matrix(f"Resources/BSDF/blinds{i:02d}.xml")
bsdf[90] = fr.load_matrix(f"Resources/glazings/0blinds.xml")

## Loop through the weather file and implement control

In [7]:
epw_path = "Resources/Delft-2022.wea"
with open(epw_path) as f:
    meta, wea = parsers.parse_wea(f.read())

## Defining the objective function for optimization
The objective is to maximize average workplane illuminance with a penalty from glare(DGP) value that are greater than 0.38. The DGP value is derived from the maximum DGP value among the eight views.
The penalty function is \
1, when DGP <=.38 \
((1 - DGP) * 1.61)^2, when DGP > 0.38

In [8]:
def objfunc(x):
    window_wpis = np.zeros(290)
    south_idx = [1, 25, 26, 27, 28, 29, 30, 31, 32]
    west_idx = [5, 6, 7, 8, 9, 10, 11, 15, 16, 17, 18, 19, 20, 35, 48]
    east_idx = [12,13, 14, 33, 34, 36, 37, 38, 39, 40, 41, 42, 43, 44, 47]
    north_idx = [2, 3, 4, 21, 22, 23, 24, 45, 46]
    for si in south_idx:
        window_wpis += fr.multiply_rgb(matrices['vmx'][si], bsdf[int(x[0]*10)], matrices['dmx'][si], _sky_mtx, weights=[47.4, 119.9, 11.6]).flatten()
    for ei in east_idx:
        window_wpis += fr.multiply_rgb(matrices['vmx'][ei], bsdf[int(x[1]*10)], matrices['dmx'][ei], _sky_mtx, weights=[47.4, 119.9, 11.6]).flatten()
    for wi in west_idx:
        window_wpis += fr.multiply_rgb(matrices['vmx'][wi], bsdf[int(x[2]*10)], matrices['dmx'][wi], _sky_mtx, weights=[47.4, 119.9, 11.6]).flatten()
    for ni in north_idx:
        window_wpis += fr.multiply_rgb(matrices['vmx'][ni], bsdf[int(x[3]*10)], matrices['dmx'][ni], _sky_mtx, weights=[47.4, 119.9, 11.6]).flatten()
    avg_wpi = window_wpis[:-4].mean()
    dgps = window_wpis[-4:] * 6.22e-5 + 0.184
    dgp = min(1, dgps.max())
    if dgp <= 0.38:
        pen = 1
    else:
        pen = ((1 - dgp) * 1.61)**2
    return avg_wpi * pen * -1

## Optimization using rbfopt (pyomo + bonmin for mixed integer non-linear programming)

In [9]:
controls = []
for w in wea:
    if w.time.hour >=8 and w.time.hour <=17:
        _sky_mtx = fr.load_matrix(fr.genskymtx([w], meta, mfactor=4, rotate=-22))
        bb = rbfopt.RbfoptUserBlackBox(4, np.array([0] * 4), np.array([9] * 4), np.array(['I'] * 4), objfunc)
        settings = rbfopt.RbfoptSettings(max_evaluations = 10) 
        alg = rbfopt.RbfoptAlgorithm(settings, bb)
        objval, x, itercount, evalcount, fast_evalcount = alg.optimize()
        controls.append((w.time, x))

  Iter  Cycle  Action             Objective value      Time      Gap
  ----  -----  ------             ---------------      ----      ---
     0      0  Initialization           -0.017313      0.16   100.00 *
     0      0  Initialization           -0.000840      0.17   100.00  
     0      0  Initialization           -0.007407      0.17   100.00  
     0      0  GlobalStep               -0.034708      0.26   100.00 *
     1      0  GlobalStep               -0.027863      0.35   100.00  
     2      0  GlobalStep               -0.035374      0.44   100.00 *
     3      0  GlobalStep               -0.034839      0.51   100.00  
     4      0  GlobalStep               -0.034703      0.60   100.00  
     5      0  AdjLocalStep             -0.003847      0.69   100.00  
     6      0  RefinementStep           -0.032639      0.76   100.00  
Summary: iters   7 evals  10 noisy_evals   0 cycles   0 opt_time    0.76 tot_time    0.76 obj        -0.035374 gap 100.00
  Iter  Cycle  Action         

     0      0  GlobalStep               -0.015341      0.29   100.00 *
     1      0  GlobalStep               -0.012930      0.37   100.00  
     2      0  GlobalStep               -0.013702      0.45   100.00  
     3      0  GlobalStep               -0.013670      0.54   100.00  
     4      0  GlobalStep               -0.011824      0.63   100.00  
     5      0  AdjLocalStep             -0.015312      0.73   100.00  
     6      0  RefinementStep           -0.013228      0.81   100.00  
Summary: iters   7 evals  10 noisy_evals   0 cycles   0 opt_time    0.81 tot_time    0.81 obj        -0.015341 gap 100.00
  Iter  Cycle  Action             Objective value      Time      Gap
  ----  -----  ------             ---------------      ----      ---
     0      0  Initialization           -0.004623      0.20   100.00 *
     0      0  Initialization           -0.000605      0.20   100.00  
     0      0  Initialization           -0.002633      0.21   100.00  
     0      0  GlobalStep     

     2      0  GlobalStep               -0.038730      0.56   100.00  
     3      0  GlobalStep               -0.037704      0.64   100.00  
     4      0  GlobalStep               -0.045454      0.70   100.00  
     5      0  AdjLocalStep             -0.045501      0.83   100.00  
     6      0  RefinementStep           -0.041011      0.93   100.00  
Summary: iters   7 evals  10 noisy_evals   0 cycles   0 opt_time    0.93 tot_time    0.93 obj        -0.045535 gap 100.00
  Iter  Cycle  Action             Objective value      Time      Gap
  ----  -----  ------             ---------------      ----      ---
     0      0  Initialization           -0.019228      0.26   100.00 *
     0      0  Initialization           -0.003430      0.26   100.00  
     0      0  Initialization           -0.012737      0.26   100.00  
     0      0  GlobalStep               -0.033732      0.41   100.00 *
     1      0  GlobalStep               -0.027062      0.50   100.00  
     2      0  GlobalStep     

     7      1  GlobalStep               -0.094657      0.82   100.00  
Summary: iters   8 evals  10 noisy_evals   0 cycles   1 opt_time    0.82 tot_time    0.82 obj        -0.101077 gap 100.00
  Iter  Cycle  Action             Objective value      Time      Gap
  ----  -----  ------             ---------------      ----      ---
     0      0  Initialization           -0.077544      0.21   100.00  
     0      0  Initialization           -0.253669      0.21   100.00  
     0      0  Initialization           -0.405255      0.22   100.00 *
     0      0  GlobalStep               -0.866258      0.28   100.00 *
     1      0  GlobalStep               -0.866015      0.36   100.00  
     2      0  Restoration phase failed.               0.41
     2      0  Changing linear system solve.           0.41
     2      1  GlobalStep               -0.305119      0.48   100.00  
     3      1  GlobalStep               -0.642251      0.55   100.00  
     4      1  GlobalStep               -0.866161   

  Iter  Cycle  Action             Objective value      Time      Gap
  ----  -----  ------             ---------------      ----      ---
     0      0  Initialization           -0.027589      0.14   100.00 *
     0      0  Initialization           -0.003053      0.14   100.00  
     0      0  Initialization           -0.015231      0.14   100.00  
     0      0  GlobalStep               -0.045074      0.27   100.00 *
     1      0  GlobalStep               -0.053966      0.36   100.00 *
     2      0  GlobalStep               -0.048360      0.45   100.00  
     3      0  GlobalStep               -0.047763      0.53   100.00  
     4      0  GlobalStep               -0.017353      0.59   100.00  
     5      0  AdjLocalStep             -0.046152      0.68   100.00  
     6      0  RefinementStep           -0.001543      0.78   100.00  
Summary: iters   7 evals  10 noisy_evals   0 cycles   0 opt_time    0.79 tot_time    0.79 obj        -0.053966 gap 100.00
  Iter  Cycle  Action         

     0      0  Initialization           -0.021695      0.18   100.00 *
     0      0  Initialization           -0.001542      0.18   100.00  
     0      0  Initialization           -0.010177      0.18   100.00  
     0      0  GlobalStep               -0.040568      0.27   100.00 *
     1      0  GlobalStep               -0.032965      0.33   100.00  
     2      0  GlobalStep               -0.008932      0.41   100.00  
     3      0  GlobalStep               -0.007204      0.51   100.00  
     4      0  GlobalStep               -0.035002      0.61   100.00  
     5      0  AdjLocalStep             -0.034314      0.73   100.00  
     6      0  RefinementStep           -0.040272      0.80   100.00  
Summary: iters   7 evals  10 noisy_evals   0 cycles   0 opt_time    0.80 tot_time    0.80 obj        -0.040568 gap 100.00
  Iter  Cycle  Action             Objective value      Time      Gap
  ----  -----  ------             ---------------      ----      ---
     0      0  Initialization 

     0      0  GlobalStep               -0.006908      0.34   100.00 *
     1      0  GlobalStep               -0.008328      0.41   100.00 *
     2      0  GlobalStep               -0.007639      0.49   100.00  
     3      0  GlobalStep               -0.007640      0.58   100.00  
     4      0  GlobalStep               -0.002521      0.64   100.00  
     5      0  AdjLocalStep             -0.007052      0.72   100.00  
     6      1  Discarded                               0.72
     7      1  GlobalStep               -0.005854      0.86   100.00  
Summary: iters   8 evals  10 noisy_evals   0 cycles   1 opt_time    0.86 tot_time    0.86 obj        -0.008328 gap 100.00
  Iter  Cycle  Action             Objective value      Time      Gap
  ----  -----  ------             ---------------      ----      ---
     0      0  Initialization           -0.024026      0.14   100.00 *
     0      0  Initialization           -0.001352      0.14   100.00  
     0      0  Initialization           -

     2      0  GlobalStep               -0.031424      0.51   100.00  
     3      0  GlobalStep               -0.031164      0.58   100.00  
     4      0  GlobalStep               -0.026691      0.66   100.00  
     5      0  AdjLocalStep             -0.034738      0.74   100.00  
     6      0  RefinementStep           -0.030828      0.86   100.00  
Summary: iters   7 evals  10 noisy_evals   0 cycles   0 opt_time    0.87 tot_time    0.87 obj        -0.034832 gap 100.00
  Iter  Cycle  Action             Objective value      Time      Gap
  ----  -----  ------             ---------------      ----      ---
     0      0  Initialization           -0.009321      0.16   100.00 *
     0      0  Initialization           -0.001295      0.16   100.00  
     0      0  Initialization           -0.005434      0.16   100.00  
     0      0  GlobalStep               -0.015497      0.28   100.00 *
     1      0  GlobalStep               -0.013091      0.36   100.00  
     2      0  GlobalStep     

     5      1  GlobalStep               -0.774472      0.85   100.00  
     6      1  AdjLocalStep             -0.774381      0.93   100.00  
Summary: iters   7 evals  10 noisy_evals   0 cycles   1 opt_time    0.93 tot_time    0.93 obj        -0.774553 gap 100.00
  Iter  Cycle  Action             Objective value      Time      Gap
  ----  -----  ------             ---------------      ----      ---
     0      0  Initialization           -0.027588      0.16   100.00 *
     0      0  Initialization           -0.005614      0.16   100.00  
     0      0  Initialization           -0.019370      0.17   100.00  
     0      0  GlobalStep               -0.045684      0.24   100.00 *
     1      0  GlobalStep               -0.040559      0.30   100.00  
     2      0  GlobalStep               -0.036851      0.40   100.00  
     3      0  GlobalStep               -0.040582      0.49   100.00  
     4      0  GlobalStep               -0.035222      0.56   100.00  
     5      0  AdjLocalStep   

  Iter  Cycle  Action             Objective value      Time      Gap
  ----  -----  ------             ---------------      ----      ---
     0      0  Initialization           -0.020372      0.27   100.00 *
     0      0  Initialization           -0.003971      0.27   100.00  
     0      0  Initialization           -0.013283      0.27   100.00  
     0      0  GlobalStep               -0.042072      0.40   100.00 *
     1      0  GlobalStep               -0.033244      0.51   100.00  
     2      0  GlobalStep               -0.033739      0.63   100.00  
     3      0  GlobalStep               -0.031645      0.74   100.00  
     4      0  GlobalStep               -0.032381      0.86   100.00  
     5      0  AdjLocalStep             -0.041981      0.99   100.00  
     6      0  RefinementStep           -0.040765      1.05   100.00  
Summary: iters   7 evals  10 noisy_evals   0 cycles   0 opt_time    1.05 tot_time    1.05 obj        -0.042072 gap 100.00
  Iter  Cycle  Action         

In [27]:
control = [c[1] for c in controls][4]
_sky_mtx = fr.load_matrix(fr.genskymtx([wea[12]], meta, mfactor=4, rotate=-22))
window_wpis = np.zeros(290)
south_idx = [1, 25, 26, 27, 28, 29, 30, 31, 32]
west_idx = [5, 6, 7, 8, 9, 10, 11, 15, 16, 17, 18, 19, 20, 35, 48]
east_idx = [12,13, 14, 33, 34, 36, 37, 38, 39, 40, 41, 42, 43, 44, 47]
north_idx = [2, 3, 4, 21, 22, 23, 24, 45, 46]
for si in south_idx:
    window_wpis += fr.multiply_rgb(matrices['vmx'][si], bsdf[control[0]*10], matrices['dmx'][si], _sky_mtx, weights=[47.4, 119.9, 11.6]).flatten()
for ei in east_idx:
    window_wpis += fr.multiply_rgb(matrices['vmx'][ei], bsdf[control[1]*10], matrices['dmx'][ei], _sky_mtx, weights=[47.4, 119.9, 11.6]).flatten()
for wi in west_idx:
    window_wpis += fr.multiply_rgb(matrices['vmx'][wi], bsdf[control[2]*10], matrices['dmx'][wi], _sky_mtx, weights=[47.4, 119.9, 11.6]).flatten()
for ni in north_idx:
    window_wpis += fr.multiply_rgb(matrices['vmx'][ni], bsdf[control[3]*10], matrices['dmx'][ni], _sky_mtx, weights=[47.4, 119.9, 11.6]).flatten()
avg_wpi = window_wpis[:-4].mean()

In [24]:
avg_wpi

0.08610722630844822

In [29]:
fr.multiply_rgb(matrices['vmx'][si], bsdf[control[0]*10], matrices['dmx'][si], _sky_mtx, weights=[47.4, 119.9, 11.6]).flatten()

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [30]:
matrix['dmx']

array([[[21.3, 21.3, 21.3]],

       [[42.3, 44.2, 49.2]],

       [[42.4, 44.4, 49.4]],

       ...,

       [[18.4, 19.3, 21.5]],

       [[18.3, 19.1, 21.3]],

       [[19.3, 20.2, 22.4]]], dtype=float32)